In [2]:
import numpy as np

In [138]:
import nipy

In [247]:
pima = np.genfromtxt('pima-indians-diabetes.data', delimiter=',')

In [248]:
def sghmc(Y, X, stogradU, M, eps, m, theta, C, V):
    n = X.shape[0]
    p = X.shape[1]
    
    # Randomly sample momentum
    r = np.random.multivariate_normal(np.zeros(M.shape[0]),M)[:,np.newaxis]
    
    # Precompute
    B = 0.5 * V * eps
    D = 2*(C-B)*eps
    Minv = np.linalg.inv(M)
    
    # Hamiltonian dynamics
    for i in range(m):
        theta = theta + (eps*np.linalg.inv(M) @ r).ravel()
        r = r - eps*stogradU(theta, Y, X, nbatch) - eps*C @ Minv @ r \
            + np.random.multivariate_normal(np.zeros(M.shape[0]),D)[:,np.newaxis]

    return(theta)

In [249]:
def stogradU(theta, Y, X, nbatch):
    '''A function that returns the stochastic gradient. Adapted from Eq. 5.
    Inputs are:
        theta, the parameters
        Y, the response
        X, the covariates
        nbatch, the number of samples to take from the full data
    '''
    alpha=5
    n = X.shape[0]
    batch_id = np.random.choice(np.arange(n),nbatch,replace=False)
    grad = -n/nbatch * X[batch_id,:].T @ (Y[batch_id][:,np.newaxis] - \
        1/(1+np.exp(-X[batch_id,:] @ theta[:,np.newaxis]))) - theta[:,np.newaxis]/alpha
    return grad/np.linalg.norm(grad)

In [250]:
nsample = 1
nbatch = 768
M = np.identity(8)
C = 1 * np.identity(8)
eps = 0.1
m = 100
V = 0 * np.identity(8)
theta = np.zeros(8)

X = pima[:,0:8]
Y = pima[:,8]

In [251]:
# w noise, no mh
samples = np.zeros((nsample, 8))

for i in range(nsample):
    theta = sghmc(Y, X, stogradU, M, eps, m, theta, C, V)
    samples[i] = theta

//anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:12: RuntimeWarning: overflow encountered in exp


In [252]:
theta

array([ 0.96127167,  0.37564097,  2.48112323, -4.62472684, -1.66040228,
       -3.79464054,  2.53358231,  2.69718489])